<a href="https://colab.research.google.com/github/akash-soni/NLP-notebooks/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment -

Use DVC or MLFLOW to convert this into a
  1. Create an API which will take the first input and it will genreate next sentences or
  2. Create an UI for this app

# Text generation with RNN

In [ ]:
import os
ROOT = "/content/drive/MyDrive/FDFS_feb_Batch/NLP/1 Text-generation"
os.chdir(ROOT)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time


In [ ]:
os.makedirs("data", exist_ok=True)

In [ ]:
!curl http://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt > "./data/shakespeare.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  42.5M      0 --:--:-- --:--:-- --:--:-- 42.5M


In [ ]:
# samething can be done using config.yaml
class Config:
  path_to_file = os.path.join("data", "shakespeare.txt")
  seq_length = 100

  Batch_size = 64
  """
  # it will keep 10000 characters of dataset in the memory(RAM),
  so while training data will be randomly picked from buffer
  and the empty space will be filled up by the data from the disk(HDD).
  Buffer is used to increase speed.
  """
  Buffer_size =10000

  """
  Letter embedding
  embedding vector for every character
  ex : for letter 'a' there is 256 size vector
       for letter 'b' there is 256 size vector
       so on...
  """
  embedding_dim = 256

  """
  From embedding layer -----> 1024 GRU Units ----> dense layer of 65(we have 65 characters)
  """
  rnn_units = 1024
  EPOCHS = 30

  checkpoint_dir = "./training_ckpt"


In [ ]:
Config.EPOCHS

30

In [ ]:
Config.path_to_file

'data/shakespeare.txt'

In [ ]:
text = open(Config.path_to_file, "rb").read().decode(encoding='utf-8')
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [ ]:
with open(Config.path_to_file) as f:
  t = f.read()


In [ ]:
t[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [ ]:
sorted(set(text))

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

# Converting text to integer vector

In [ ]:
#get all the unique characters in sorted order
vocab = sorted(set(text))
print("no. of uniqe characters are ", len(vocab))

no. of uniqe characters are  65


In [ ]:
#enumerate all the characters to integers
char2idx = {uniChar : idx for idx, uniChar in enumerate(vocab)}

In [ ]:
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [ ]:
text

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [ ]:
# creating word text vector
text_as_int = np.array([char2idx[c] for c in text])

text_as_int # all the text is now represented as integer

array([18, 47, 56, ..., 45,  8,  0])

In [ ]:
len(text_as_int)

1115394

In [ ]:
text[:13], text_as_int[:13]

('First Citizen', array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]))

# Get the max sequence length

In [ ]:
print("total length is : ", len(text))

total length is :  1115394


In [ ]:
examples_per_epoch = len(text)//(Config.seq_length + 1) # +1 is for the last sentence
examples_per_epoch

11043

In [ ]:
idx2char = {val:key for key, val in char2idx.items()}

In [ ]:
#slicing each converted character of dataset to integer
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i


In [ ]:
# combining each tensors according to the sequence length
#  Total characters  : 1115394
#  Batch size : 101
#  Total Batches : 1115394//101 = 11043
sequences = char_dataset.batch(Config.seq_length+ 1, drop_remainder = True)
# take
for item in sequences.take(5):
  print("\nSEQUENCE")
  print(repr("".join([idx2char[c] for c in item.numpy()])))


SEQUENCE
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '

SEQUENCE
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'

SEQUENCE
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"

SEQUENCE
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"

SEQUENCE
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


# creating a target value

Ex : First Citize -> irst Citizen

In [ ]:
# creating input and output chunks for each individual sequence
from numpy.lib.shape_base import split

def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset =  sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print("INPUT DATA:\n")
  print(repr("".join([idx2char[i] for i in input_example.numpy()])))

  print("\n\nTARGET_DATA:\n")
  print(repr("".join([idx2char[i] for i in target_example.numpy()])))

INPUT DATA:

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'


TARGET_DATA:

'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# Creating training Batches

In [ ]:
dataset =  dataset.shuffle(Config.Batch_size).batch(Config.Batch_size, drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
vocab

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
# finidng no. of output classes
Config.vocab_size = len(vocab)
Config.vocab_size

65

# Building a Model

In [ ]:
def build_model(
    vocab_size ,embedding_dim, rnn_units, batch_size
    ):
    model = tf.keras.Sequential([
                                 # vocab_size is input of 65 characters
                                 # embedding_dim is output vector of 256 dimension
                                 # batch_input_shape is batches of 64 will be passed
                                 tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
                                 tf.keras.layers.GRU(rnn_units,
                                                     return_sequences = True,
                                                     stateful = True,
                                                     recurrent_initializer = 'glorot_uniform'
                                                     ),
                                 tf.keras.layers.Dense(vocab_size)
                                 ])
    return model

In [ ]:
model = build_model(
    vocab_size= Config.vocab_size,
    embedding_dim=Config.embedding_dim,
    rnn_units = Config.rnn_units,
    batch_size = Config.Batch_size
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# defining loss
# logits : means we are taking raw output as there is no activation function like a softmax layer
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_prefix = os.path.join(Config.checkpoint_dir, "chkpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

# Compiling model

In [ ]:
history = model.fit(dataset, epochs = Config.EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 21s 67ms/step - loss: 2.7229
Epoch 2/30
172/172 [==============================] - 11s 63ms/step - loss: 2.0446
Epoch 3/30
172/172 [==============================] - 11s 66ms/step - loss: 1.8017
Epoch 4/30
172/172 [==============================] - 11s 61ms/step - loss: 1.6372
Epoch 5/30
172/172 [==============================] - 10s 60ms/step - loss: 1.5267
Epoch 6/30
172/172 [==============================] - 10s 60ms/step - loss: 1.4513
Epoch 7/30
172/172 [==============================] - 10s 58ms/step - loss: 1.3945
Epoch 8/30
172/172 [==============================] - 10s 59ms/step - loss: 1.3495
Epoch 9/30
172/172 [==============================] - 10s 59ms/step - loss: 1.3087
Epoch 10/30
172/172 [==============================] - 10s 61ms/step - loss: 1.2704
Epoch 11/30
172/172 [==============================] - 10s 60ms/step - loss: 1.2328
Epoch 12/30
172/172 [==============================] - 11s 61ms/step - loss: 1.1940
E

# Loading the model from checkpoint

In [ ]:
# Restoring checkpoint -

tf.train.latest_checkpoint(Config.checkpoint_dir)

'./training_ckpt/chkpt_30'

In [ ]:
# building a model from restored checkpoint
model_from_ckpt = build_model(
    vocab_size= Config.vocab_size,
    embedding_dim=Config.embedding_dim,
    rnn_units = Config.rnn_units,
    batch_size = 1
)

In [ ]:
model_from_ckpt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# loading the model weights
model_from_ckpt.load_weights(tf.train.latest_checkpoint(Config.checkpoint_dir))

model_from_ckpt.build(tf.TensorShape([1,None]))

In [ ]:
model_from_ckpt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


# Prediction

In [ ]:
# prediction function
def generate_text(model, start_string, no_of_chars_to_gen = 1000):
  # convert the input text to nos.
  input_val = [char2idx[s] for s in start_string] # text converted to int
  input_val = tf.expand_dims(input_val, 0) # [x] -> [1, x]

  text_generated = list()

  temperature = 1.0 # normalization factor for output

  model.reset_states() # Resetting the previous states if any while predictions.
  for i in range(no_of_chars_to_gen):
    predictions = model(input_val)

    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature

    print(predictions)
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
    print(predicted_id)

    input_val = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return start_string + "".join(text_generated)


In [ ]:
generate_text(model = model_from_ckpt, start_string="ROMEO", no_of_chars_to_gen=1000)

Streaming output truncated to the last 5000 lines.
46
tf.Tensor(
[[ -0.06653299   1.2764052   -3.474028    -6.264572    -6.245648
   -3.3355303    0.92012966   0.36543703  -3.0461154   -8.147454
   -2.3949435   -2.1826947   -4.1597757    0.8801986   -2.865833
  -11.789806    -4.2326527    1.5726764   -9.499927    -8.733558
   -7.614655     1.3536208   -3.489345    -9.21428     -9.990447
   -6.4616804   -5.780339     0.8174836   -7.7788687   -7.3789043
   -1.6313525   -7.2842736   -7.5759587   -1.414701   -13.950598
   -5.1147056   -8.107185    -4.789936    -7.1240683   11.641666
    3.4992452   -2.234749    -5.96805      8.138593    -5.5595484
   -6.2228284   -2.3761902   10.248687    -9.579624    -6.254823
    0.4879753   -0.6341859   -1.0683044    9.255517    -4.7372727
   -4.6562605    6.934563    -2.1788538    3.1671798    7.0967717
   -6.7015257    1.200093    -4.725007     7.9483237   -4.335621  ]], shape=(1, 65), dtype=float32)
47
tf.Tensor(
[[ -9.057309    -5.4273114   -8.70572

"ROMEO:\nThe sister nook in law:\nNeither, let us so, 'tis books! and what is he?\n\nProvost:\nNo, my ove or free.\n\nKATHARINA:\nPith'd too, and go do we his queen?\n\nHORTENSIO:\nAnd teach,\nYet can my master go wash my looking: should I see this brother Gilde of it.\n\nPage:\nNow, my horse, my husband, being rewitt.\n\nVOLUMNIA:\nOn sir, unto differd, Grumio!\n\nPage:\nAnd a dispatch: I know their fatal call him a hundred comfort comeo: sir?\n\nBRAKENBAL:\nWhy, not a jad with eage?\n\nGRUMIO:\n\nKATHARINA:\nI know the wanton still custommer. Petruchio is tence, or say to take\nHery shoulder-Money, the king's hid,\nFarewell, good Aundier!\nHow now! why, name my daughter knows our personague.\nThis is the private; crakest thou pershape, Richard, thrice with\nsaddless moon, madam: 'tis not my mistress.\n\nPETRUCHIO:\nI must inquiry ock;\nTo could relish'd?\n\nKATHARINA:\nFear not mock my vow:\nIf I live to know his horse, beside to make\nAs many for the greater stoopiec.\n\nPETRUCHIO:\

In [ ]:
generate_text(model = model_from_ckpt, start_string="First Citizen", no_of_chars_to_gen=1000)

Streaming output truncated to the last 5000 lines.
[[ -0.9962001    5.592655    -0.8184795   -8.644028   -11.538087
    3.1289566    1.9022342   -2.3576088   -0.1401355   -7.4996457
   -2.0325828   -3.2417886   -3.563673    -4.8046947   -5.1802964
   -5.668945    -2.7400146   -6.0401716   -8.244292    -2.4848971
   -6.8505526   -2.0076463  -10.039997    -6.5799003   -7.348824
   -8.771597    -3.7145572   -7.4543176   -6.7011347   -5.559804
   -5.947311    -4.904744    -7.9127264   -5.544271    -6.80132
   -4.844918   -11.886371    -7.951801    -4.09219      2.3551214
   -2.4381447   -2.4062078    6.150531    13.816967    -0.07689253
    4.225004    -2.0840728   -2.2783046   -1.4962139   -4.3324833
   -1.6089126   -4.9620395    2.1243827    1.8221408   -4.359805
   -1.9898568   -2.6998346    6.650661     2.8817835   -0.8862292
    0.64712495  -0.98864836  -5.578026    -2.1741776   -2.5198526 ]], shape=(1, 65), dtype=float32)
43
tf.Tensor(
[[ 2.1046958  10.992213    4.2327933  -6.374407 

"First Citizen:\nGo, profess, to say the father,\nHer father was the wanton should; 'tis right.\nCAMILLO:\nNo sin! To court hither.\n\nKING EDWARD IV:\nWARWICK:\nWho then came in a frettwould call thee shrive\nTo every of your taken and hide;\nThe shruds and chrint and not cheak ed warman?\n\nBIONDELLO:\nInfect I have dispatch'd in me?\nThen, good master, musicinut or honour:\nThen will I rear thou know; behold him, And now, te'll death to\ngrant it counsel: thou darest not fall, which on the day,\nThat gave at one come?\n\nPETRUCHIO:\nI pray you do, to have at Editio.\n\nBRUTUS:\nAnd when a nature may hear.\n\nPETRUCHIO:\nA Placey mooon.\n\nKATHARINA:\nThere's no knowledge she minds:\nThe were gone, sir, Peter upon your honour.\n\nMARCIUS:\nGo to\nLoddand that I may, sir.\n\nPETRUCHIO:\nWhip, fie! 'tis like to bed.\nWe joor some brief, with foods, this mignty,\nWho climis more behind, or would appreased\nThe bourn so speak; say with thee to be sure my church o'erch-best,\nAnd riving t

In [ ]:
for _ in range(15):
  predictions = [[1.,2.,3.,4.]]
  samples = tf.random.categorical(predictions, 1)[-1,0]
  print(samples.numpy())

3
3
3
3
3
3
2
1
3
3
3
3
3
3
0


in the above numbers are varying between 0,1,2 and 3

now when we introduce a very high number the prediction is always the index of highest number

In [ ]:
for _ in range(15):
  predictions = [[1.,2.,3.,4.,55.]]
  samples = tf.random.categorical(predictions, 1)[-1,0]
  print(samples.numpy())

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


when we introduce 56 which is nearby 55, the prediction vary between 4 and 5 but we will never get 0,1,2,3

In [ ]:
for _ in range(15):
  predictions = [[1.,2.,3.,4.,55.,56.]]
  samples = tf.random.categorical(predictions, 1)[-1,0]
  print(samples.numpy())

5
5
5
4
4
4
5
5
5
4
4
5
5
5
4


let's introduce 100 and 101 then we will get index between 6 and 7

In [ ]:
for _ in range(15):
  predictions = [[1.,2.,3.,4.,55.,56.,100.,101., 1001]]
  samples = tf.random.categorical(predictions, 1)[-1,0]
  print(samples.numpy())

8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
